参考资料：

https://blog.csdn.net/dark_scope/article/details/17228643

In [2]:
data = [[1, 1, 1, 0, 0],
        [2, 2, 2, 0, 0],
        [1, 1, 1, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 0, 2, 2],
        [0, 0, 0, 3, 3],
        [0, 0, 0, 1, 1]]

In [8]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=6)

# 可以看到，前 3 个数值比其它的值大了很多
U, Sigma, VT = np.linalg.svd(data)
Sigma

array([9.7214  , 5.293979, 0.684226, 0.      , 0.      ])

<b><font size='3' color='ff0000'>可以把后两个奇异值砍掉，这就相当于，砍掉了 U 矩阵的后两列，和 V 矩阵的后两个。</font></b>

![](./SVD 示意图.jpg)

接下来重构原始矩阵。

In [10]:
U[:, :3].dot(np.diag(Sigma[:3])).dot(VT[:3])

array([[ 1.,  1.,  1., -0., -0.],
       [ 2.,  2.,  2.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.],
       [ 5.,  5.,  5., -0., -0.],
       [ 1.,  1., -0.,  2.,  2.],
       [ 0.,  0., -0.,  3.,  3.],
       [ 0.,  0., -0.,  1.,  1.]])

再看看原始矩阵，几乎没有丢失任何信息。<b><font size='3' color='ff0000'>其实 PCA 去噪声和 SVD 用于降噪的方法是一样的，PCA 去掉了特征值比较小的那些特征向量，SVD 去掉了奇异值比较小的 U 和 V 矩阵中的对应向量。</font></b>

In [11]:
data

[[1, 1, 1, 0, 0],
 [2, 2, 2, 0, 0],
 [1, 1, 1, 0, 0],
 [5, 5, 5, 0, 0],
 [1, 1, 0, 2, 2],
 [0, 0, 0, 3, 3],
 [0, 0, 0, 1, 1]]

## SVD 用于推荐引擎

# 基于物品的推荐


![](https://img-blog.csdn.net/20131210170818343)

我是这样理解的：

1、用户至上，所以用户在第 1 维度，即每一行表示一个用户的数据；
2、基于物品，物品是在列的角度，所以看列，比较的是列的相似度；
3、具体的方法是这样的：<b><font size='3' color='ff0000'>在用户已经评价过的每一个商品里，都计算和这个待推荐的商品的相似度，只取同时非零的两列放在一起计算相似度，并且相似度和评分要进行加权求和，才是用户可能给这个商品的打分。</font></b>



In [37]:
import numpy as np

In [38]:
def eclud_sim(vec1, vec2):
    """
    基于欧式距离计算相似度
    :param vec1: 假定是列向量
    :param vec2: 假定是列向量
    :return:
    """
    return 1.0 / (1.0 + np.linalg.norm(vec1 - vec2))


def pears_sim(vec1, vec2):
    """
    会检查是否存在 3 个或更多的点。
    :param vec1:
    :param vec2:
    :return:
    """
    # 如果不存在，该函数返回 1.0，此时两个向量完全相关。
    if len(vec1) < 3:
        return 1.0
    # corrcoef 直接计算皮尔逊相关系数，范围 [-1, 1] ，归一化后 [0, 1]
    return 0.5 + 0.5 * np.corrcoef(vec1, vec2, rowvar=False)[0][1]


def cos_sim(vec1, vec2):
    """
    # 计算余弦相似度，如果夹角为 90 度，相似度为 0；
    # 如果两个向量的方向相同，相似度为 1.0
    :param vec1:
    :param vec2:
    :return:
    """
    num = float(vec1.T * vec2)
    denom = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    return 0.5 + 0.5 * (num / denom)

In [39]:
def stand_est(data_mat, user_id, similar_calc_method, item_id):
    """
    基于物品相似度的推荐引擎
    计算某用户未评分物品中，以对该物品和其他物品评分的用户的物品相似度，然后进行综合评分
    :param data_mat: 训练数据集
    :param user_id: 单个用户的编号
    :param similar_calc_method: 相似度计算方法
    :param item_id: 未评分的物品编号
    :return: 评分（0～5之间的值）
    """
    # 得到数据集中的物品数目
    n = np.shape(data_mat)[1]
    # 初始化两个评分值
    sim_total = 0.0
    rat_sim_total = 0.0
    # 遍历某一行中的每个物品（对用户评过分的物品进行遍历，并将它与其他物品进行比较）
    for j in range(n):
        # 单个用户对某个物品的评分
        user_rating = data_mat[user_id, j]

        if user_rating == 0:
            # 用户评分为 0 ，表示用户没有评价这个商品，跳过
            continue

        # 【寻找两个用户都评级的物品】
        # logical_and 计算 x1 和 x2 元素的真值，同时为 True，结果才为 True
        # over_lap 表示两列中，即两个物品，都被评分的行索引编号
        over_lap = np.nonzero(np.logical_and(
            data_mat[:, item_id].A > 0, data_mat[:, j].A > 0))[0]
        # 如果相似度为 0 ，则两着没有任何重合元素，终止本次循环
        if len(over_lap) == 0:
            similarity = 0
        # 如果存在重合的物品，则基于这些重合物重新计算相似度。
        else:
            # dataMat[over_lap, item] 和 dataMat[over_lap, j] 是两个向量
            # 两个向量才可以计算相似度
            # 【都评分的两个向量，两个列，才计算相似度】
            similarity = similar_calc_method(
                data_mat[over_lap, item_id], data_mat[over_lap, j])

        # 相似度会不断累加，每次计算时还考虑相似度和当前用户评分的乘积
        # user_rating 用户已经评价过的商品有评分
        sim_total += similarity
        rat_sim_total += similarity * user_rating
    if sim_total == 0:
        return 0
    else:
        # 归一化
        # 通过除以所有的评分总和，对上述相似度评分的乘积进行归一化，使得最后评分在 0~5 之间，
        # 这些评分用来对预测值进行排序
        return rat_sim_total / sim_total

In [40]:
# 【推荐引擎】默认调用 stand_est()函数，产生了最高的 N 个推荐结果
# 如果不指定 N 的大小，则默认值为3。该函数另外的参数还包括相似度计算方法和估计方法
def recommend(data_mat, user_id, N=3, similar_calc_method=cos_sim, est_method=stand_est):
    """

    :param data_mat: 训练数据集
    :param user_id: 用户编号
    :param N:
    :param similar_calc_method: 相似度计算方法
    :param est_method: 使用的推荐算法
    :return: 返回最终 N 个推荐结果
    """

    # 寻找未评级的物品
    # 对给定的用户建立一个未评分的物品列表
    print(user_id)

    unrated_items = np.nonzero(data_mat[user_id, :].A == 0)[1]
    # 如果不存在未评分物品，那么就退出函数
    if len(unrated_items) == 0:
        return 'you rated everything'
    # 物品的编号和评分值
    item_scores = []
    # 对这个用户的所有的没有评分的商品进行预估评分
    for item in unrated_items:
        # 获取 item 该物品的评分
        estimated_score = est_method(
            data_mat, user_id, similar_calc_method, item)
        item_scores.append((item, estimated_score))
    # 按照评分得分 进行逆排序，获取前N个未评级物品进行推荐
    return sorted(item_scores, key=lambda x: x[1], reverse=True)[: N]




In [34]:
# 基于内容的推荐引擎示例矩阵
item_bases_example = np.mat([[4, 4, 0, 2, 2],
                             [4, 0, 0, 3, 3],
                             [4, 0, 0, 1, 1],
                             [1, 1, 1, 2, 0],
                             [2, 2, 2, 0, 0],
                             [1, 1, 1, 0, 0],
                             [5, 5, 5, 0, 0]])



2


[(2, 2.5), (1, 2.0243290220056256)]

In [41]:
# 使用余弦相似度
recommend(data_mat=item_bases_example, user_id=2)

2


[(2, 2.5), (1, 2.0243290220056256)]

In [43]:
# 使用欧氏距离
recommend(data_mat=item_bases_example, user_id=2,
          similar_calc_method=eclud_sim)

2


[(2, 3.0), (1, 2.8266504712098603)]

In [44]:
# 使用皮尔逊相关系数
recommend(data_mat=item_bases_example, user_id=2,
          similar_calc_method=pears_sim)

2


[(2, 2.5), (1, 2.0)]

## 利用 SVD 提高推荐的效果

In [46]:
# 利用 SVD 提高推荐的效果，菜肴矩阵
svd_mat = np.mat([[2, 0, 0, 4, 4, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5],
                  [0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0],
                  [3, 3, 4, 0, 3, 0, 0, 2, 2, 0, 0],
                  [5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
                  [4, 0, 4, 0, 0, 0, 0, 0, 0, 0, 5],
                  [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4],
                  [0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0],
                  [0, 0, 0, 3, 0, 0, 0, 0, 4, 5, 0],
                  [1, 1, 2, 1, 1, 2, 1, 0, 4, 5, 0]])

In [49]:
U, Sigma, VT = np.linalg.svd(svd_mat)
Sigma

array([13.434282, 11.819083,  8.201761,  6.869125,  5.29063 ,  3.912136,
        2.945625,  2.354861,  2.087021,  0.708716,  0.      ])

In [57]:
Sigma_power = Sigma**2
Sigma_power

array([180.479931, 139.690727,  67.26888 ,  47.184876,  27.990768,
        15.304805,   8.676707,   5.545372,   4.355656,   0.502278,
         0.      ])

In [63]:
sum(Sigma_power[:5])/sum(Sigma_power)

0.930815254585099

In [64]:
import numpy as np


def svd_est(data_mat, user_id, similar_calc_method, item_id):
    """
    # 基于 SVD 的评分估计
    # 基于内容的评分矩阵中，用户信息被压缩，去掉的是那些奇异值比较低的特征向量
    # 这里用到的是左奇异矩阵
    # 在 recommend() 中，这个函数用于替换对 stand_est() 的调用，该函数对给定用户给定物品构建了一个评分估计值

    :param data_mat: 训练数据集
    :param user_id: 用户编号
    :param similar_calc_method: 相似度计算方法
    :param item_id: 未评分的物品编号
    :return: rat_sim_total/sim_total，评分（0～5之间的值）
    """
    # 物品数目
    n = np.shape(data_mat)[1]
    # 对数据集进行 SVD 分解
    sim_total = 0.0
    rat_sim_total = 0.0
    # 奇异值分解
    # 在SVD分解之后，我们只利用包含了90%能量值的奇异值，这些奇异值会以NumPy数组的形式得以保存
    U, Sigma, VT = np.linalg.svd(data_mat)

    # # 分析 Sigma 的长度取值
    # analyse_data(Sigma, 20)

    # 如果要进行矩阵运算，就必须要用这些奇异值构建出一个对角矩阵
    Sig4 = np.mat(np.eye(4) * Sigma[: 4])

    # 利用 U 矩阵将物品转换到低维空间中，构建转换后的物品(物品+4个主要的特征)
    xformed_items = data_mat.T * U[:, :4] * Sig4.I

    # 对于给定的用户，for 循环在用户对应行的元素上进行遍历
    # 这和 standEst() 函数中的for循环的目的一样，只不过这里的相似度计算时在低维空间下进行的。
    for j in range(n):
        user_rating = data_mat[user_id, j]
        if user_rating == 0 or j == item_id:
            continue
        # 相似度的计算方法也会作为一个参数传递给该函数
        similarity = similar_calc_method(xformed_items[item_id, :].T, xformed_items[j, :].T)
        # for 循环中加入了一条print语句，以便了解相似度计算的进展情况。如果觉得累赘，可以去掉
        print('the %d and %d similarity is: %f' % (item_id, j, similarity))
        # 对相似度不断累加求和
        sim_total += similarity
        # 对相似度及对应评分值的乘积求和
        rat_sim_total += similarity * user_rating
    if sim_total == 0:
        return 0
    else:
        # 计算估计评分
        return rat_sim_total / sim_total


In [67]:
# 使用余弦相似度
recommend(data_mat=item_bases_example, user_id=2, est_method=svd_est)

2
the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 2.253270755977714), (1, 1.9921514636756923)]

In [68]:
recommend(data_mat=item_bases_example, user_id=2,
          est_method=svd_est, similar_calc_method=pears_sim)

2
the 1 and 0 similarity is: 0.626075
the 1 and 3 similarity is: 0.672793
the 1 and 4 similarity is: 0.614375
the 2 and 0 similarity is: 0.429334
the 2 and 3 similarity is: 0.387057
the 2 and 4 similarity is: 0.043539


[(2, 2.497798373616035), (1, 1.9816972841840976)]

In [69]:
def analyse_data(Sigma, loop_num=20):
    """
    分析 Sigma 的长度取值
    :param Sigma: Sigma 的值
    :param loop_num: 循环次数
    :return: 
    """

    # 总方差的集合（总能量值）
    Sig2 = Sigma ** 2
    SigmaSum = sum(Sig2)
    for i in range(loop_num):
        SigmaI = sum(Sig2[:i + 1])
        # 根据自己的业务情况，就行处理，设置对应的 Singma 次数

        # 通常保留矩阵 80% ～ 90% 的能量，就可以得到重要的特征并取出噪声。
        print('主成分：%s, 方差占比：%s%%' % (format(i + 1, '2.0f'), format(SigmaI / SigmaSum * 100, '4.2f')))


In [76]:
import numpy as np

def img_load_data(filename):
    """
    图像压缩函数，加载并转换数据
    :param filename: 
    :return: 
    """
    myl = []
    # 打开文本文件，并从文件以数组方式读入字符
    for line in open(filename).readlines():
        new_row = []
        for i in range(32):
            new_row.append(int(line[i]))
        myl.append(new_row)
    # 矩阵调入后，就可以在屏幕上输出该矩阵
    my_mat= np.mat(myl)
    return my_mat


def print_mat(in_mat, thresh=0.8):
    """
    打印矩阵
    :param in_mat: 
    :param thresh: 
    :return: 
    """
    # 由于矩阵保护了浮点数，因此定义浅色和深色，遍历所有矩阵元素，当元素大于阀值时打印 1 ，否则打印0
    for i in range(32):
        for k in range(32):
            if float(in_mat[i, k]) > thresh:
                print(1,end='')
            else:
                print(0,end='')
        print('')


# 实现图像压缩，允许基于任意给定的奇异值数目来重构图像
def img_compress(num_sv=3, thresh=0.8):
    """
    
    :param num_sv: Sigma 长度
    :param thresh: 判断的阈值
    :return: 
    """
    # 构建一个列表
    my_mat = img_load_data('./0_5.txt')

    print("****original matrix****")
    # 对原始图像进行SVD分解并重构图像e
    print_mat(my_mat, thresh)

    # 通过Sigma 重新构成SigRecom来实现
    # Sigma是一个对角矩阵，因此需要建立一个全0矩阵，然后将前面的那些奇异值填充到对角线上。
    U, Sigma, VT = np.linalg.svd(my_mat)
    # sig_recon = mat(zeros((numSV, numSV)))
    # for k in range(numSV):
    #     sig_recon[k, k] = Sigma[k]

    # 分析插入的 Sigma 长度
    analyse_data(Sigma, 20)

    sig_recon = np.mat(np.eye(num_sv) * Sigma[: num_sv])
    reconMat = U[:, :num_sv] * sig_recon * VT[:num_sv, :]
    print("****reconstructed matrix using %d singular values *****" % num_sv)
    print_mat(reconMat, thresh)

In [77]:
# 压缩图片
img_compress(2)

****original matrix****
00000000000000110000000000000000
00000000000011111100000000000000
00000000000111111110000000000000
00000000001111111111000000000000
00000000111111111111100000000000
00000001111111111111110000000000
00000000111111111111111000000000
00000000111111100001111100000000
00000001111111000001111100000000
00000011111100000000111100000000
00000011111100000000111110000000
00000011111100000000011110000000
00000011111100000000011110000000
00000001111110000000001111000000
00000011111110000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000011111100000000001111000000
00000001111100000000001111000000
00000001111100000000011111000000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000001111100000
00000000111110000000011111000000
00000000111110000000111111000000
00000000111111000001111110000000
00000000011111111111111110000000
00000000001111111111111110000000
00000000001111111111111110000000
0000000000011111111